In [ ]:
import pandas as pd
from pyprojroot import here

import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv(here() / 'data/base/data.csv')

In [ ]:
a = data.intervention_binary.value_counts()

In [ ]:
series = {
    'Calling in \n additional staff' : data.intervention_categorical___0.sum(),
     'Open additional \n inpatient beds' : data.intervention_categorical___1.sum(),
     'Adjust decision \n making in triage' : data.intervention_categorical___2.sum(),
    'Communicate \n catchment area to \n limit input' : data.intervention_categorical___3.sum(),
    'Other' : data.intervention_categorical___4.sum()
    }

series = pd.Series(series)
series = ((series / data.shape[0]) * 100).round(0)

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))

series.plot.barh(
    ax=ax, 
    color='lightgrey',
    width=.6)

ax.set_xlim([0,100])

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.savefig(here() / 'output/plots/interventions.png', dpi=300, bbox_inches='tight')